# Résolution du jeu de Morpions par reinforcement learning

Ce projet à pour but de comparer et essayer différent algorithme de reinforcement learning autour du jeu du Morpions.

L'agent est entrainé pour joué en tant que premier joueur.

Les états sont représentés pour une liste de 9 éléments. \\
0 -> Case vide \\
1 -> Croix (premier joueur) \\
-1 -> Rond (deuxième joueur) \\
E = [0 , 0 ,0 , 1 , -1 , 0 , 1 , -1 , 0] -> "image de morpions"


Les seuls états présents sont ceux jouable pour le joueur 1  

L'ensemble des actions d'un états est une liste d'indice, correspondant au indice libre de l'état

"image morpions" -> [liste d'actions]

L'état 𝐒{n+1} se déduit de 𝐒_n et 𝐴_n.
Pour ce faire, on joue d'abord un coup a valable, soit l'état donnée est terminal est donc vaut S_n+1, soit l'état n'est pas terminal et on choisit un coup valable de l'adversaire pour calculer S_n+1

R_n vaut 10 si S_n est gagnant pour le joueur 1. \\
R_n vaut -10 si S_n est gagnant pour le joueur 2. \\
R_n vaut 0 si S_n est nul.

In [ ]:
import numpy as np
import random

## Définition de l'ensemble des états 𝐒

La première partie de ce code est consacré à la définiton de l'ensemble 𝐒, qui contient l'ensemble des états, ainsi qu'a de nombreuse fonctions sur les états.



In [ ]:
'''
E : un état
renvoie la liste des actions possibles pour cet état
'''
def list_actions(E):
  a = []
  for i in range(len(E)):
    if E[i] == 0:
      a.append(i)
  return a

In [ ]:
'''
E : un état
a : une action
J : un joueur (1 ou -1)
renvoie l'état obtenu après l'action
'''
def etat_action(E,a,J):
  F = E.copy()
  F[a] = J
  return F

In [ ]:
'''
E : un état
i : indice dans {0,1,2}
renvoie la ligne i de l'état E
'''
def ligne(E,i):
  return E[i*3:i*3+3]
'''
E : un état
i : indice dans {0,1,2}
renvoie la collumns i de l'état E
'''
def collumns(E,i):
  return E[i::3]
'''
E : un état
i : indice dans {0,1}
renvoie la diagonale i de l'état E
'''
def diag(E,i):
  if i == 0:
    return E[0::4]
  if i == 1:
    return E[2:7:2]

In [ ]:
'''
E : un état
renvoie True si l'un des joueur a gagné , False sinon
'''
def etat_final(E):
  for i in range(3):
    if np.abs(np.sum(ligne(E,i))) == 3:
      return True
    if np.abs(np.sum(collumns(E,i))) == 3:
      return True
  for i in range(2):
    if np.abs(np.sum(diag(E,i))) == 3:
      return True
  return False

'''
E : un état
renvoie True si l'un des joueur a gagné ou si il y a égalité, False sinon
'''
def etat_terminal(E): # on a gagné ou match nul
  return etat_final(E) or (len(list_actions(E)) == 0)

In [ ]:
def print_etat(S):
  print("")
  print(S[0] , S[1] , S[2] ,"           0 1 2")
  print(S[3] , S[4] , S[5] ,"           3 4 5")
  print(S[6] , S[7] , S[8] ,"           6 7 8")
  print("--------")

In [ ]:
'''
Renvoie l'ensemble des états S avec un parcours en profondeur depuis l'état initial
'''
def ensemble_etat():
  E = [0,0,0,
       0,0,0,
       0,0,0]
  S = [E]
  AVisiter = [E]
  while(len(AVisiter) != 0):
    e = AVisiter[0]
    AVisiter.remove(e)
    actions = list_actions(e)
    if etat_terminal(e):
      S.append(e)
    else:
      for a in actions:
        F = etat_action(e,a,1)
        if etat_terminal(F):
          S.append(F)
        else:
          actions_F = list_actions(F)
          for a_f in actions_F:
            G = etat_action(F,a_f,-1)
            AVisiter.append(G)
            S.append(G)
  return S

In [ ]:
'''
Renvoie l'état d'indice x dans S
'''
def numero_etat(x):
  return S[x]
'''
Renvoie l'indice de l'état E dans S
'''
def etat_numero(e,S):
  i = 0
  for s in S:
    if s == e:
      return i
    i += 1
  return i-1

In [ ]:
S = ensemble_etat()

## Algorithme d'apprentisage

In [ ]:
'''
E : un état
pi : une matrice de probabilités
renvoie l'action choisit selon pi pour l'état E
'''
def predict(E, pi): #
  probabilities = pi[etat_numero(E,S)]
  return random.choices(range(len(probabilities)), weights=probabilities, k=1)[0]

In [ ]:
'''
a : une action
E : un état
renvoie l'état obtenu après l'action et la récompense associé à l'état obtenu
10 si on gagne
-10 si on perd
0 sinon
'''
def execute(a,E):
  F = etat_action(E,a,1) # notre tour de jouer
  if etat_final(F):
    return 10 , F
  if etat_terminal(F):
    return 0 , F

  actions = list_actions(F)
  probabilities = []
  probabilities = [1 / len(actions) for i in range(len(actions))]
  indice = random.choices(range(len(probabilities)), weights=probabilities, k=1)[0]
  a_prime = actions[indice]
  E_prime = etat_action(F,a_prime,-1) # tour de l'adversaire
  if etat_final(E_prime):
    return -10 , E_prime

  return 0 , E_prime

In [ ]:
def controle(pi,E,Q,epsilon):
  actions = list_actions(E)
  numero = etat_numero(E,S)
  # on vérifie que Q possède un max non nul
  ok = False
  for x in Q[numero]:
    if x > 0:
      ok = True
  if ok:
    argmx = np.argmax(Q[numero])
    for a in actions:
      if a == argmx:
        pi[numero][a] = (epsilon/len(actions)) + 1 - epsilon
      else:
        pi[numero][a] = epsilon/len(actions)
  return pi

###Monte-Carlo

In [ ]:
'''
l : une liste d'entier
n : un entier
renvoie la somme des n derniers éléments de la liste
'''
def sum(l,n):
  x = 0
  for i in range(n):
    x += l[len(l) - 1 - i]
  return x

In [ ]:
def Monte_Carlo(pi,Q,epsilon = 0.1,k = 1000,n = 500):
  for i in range(k):
    if i > n:
      epsilon = epsilon/(i-n+1)
    E = [0,0,0,
        0,0,0,
        0,0,0]
    S_parcourue = [E]
    a_joue = []
    R_moy = []
    # on joue une partie pour recolter les R par rapport à chaque couple (E,a)
    while not(etat_terminal(E)):
      a = predict(E,pi)
      a_joue.append(a)
      R , E = execute(a,E)
      S_parcourue.append(E)
      R_moy.append(R)

    # On attribue à chaque couple (E,a) la moyenne des returns
    for j in range(len(a_joue)):
      Q[etat_numero(S_parcourue[j],S)][a_joue[j]] = sum(R_moy,len(a_joue) - j)/(len(a_joue) - j)
    pi = controle(pi,E,Q,epsilon)
  return pi , Q

###Sarsa

In [ ]:
def Sarsa(pi,Q,alpha = 0.2,gamma = 0.99,epsilon = 0.1,k = 1000, n = 500):
  for i in range(k):
    if i > n:
      epsilon = epsilon/(i-n+1)
    E = [0,0,0,
        0,0,0,
        0,0,0]
    a = predict(E,pi)
    while not(etat_terminal(E)):
      R , E_prime = execute(a,E)
      if len(list_actions(E_prime)) != 0:
        a_prime = predict(E_prime,pi)
        Q[etat_numero(E,S)][a] = Q[etat_numero(E,S)][a] + alpha*(R + gamma*Q[etat_numero(E_prime,S)][a_prime] - Q[etat_numero(E,S)][a])
        pi = controle(pi,E,Q,epsilon)
      E = E_prime
      a = a_prime
  return pi , Q

## Définition de π et de 𝑸

In [ ]:
'''
Pi est un tabeleau numpy qui ne sera jamais modifiée durant l'entrainement
P[i][a] = probabilité de l'action a dans l'état S[i] (repartie de manière uniforme)
Q[i][a] = valeur de l'action a dans l'état S[i] (initialisé à 0)
'''
Pi = np.zeros([len(S),9])
for i in range(len(S)):
  for a in list_actions(S[i]):
    Pi[i][a] = 1/len(list_actions(S[i]))
Q = np.zeros((len(S),9))

## Entrainement sur différent algorithme

In [ ]:
'''
k : le nombre d'itération de notre modèle
n : le nombre d'itération avant de diminuer epsilon
'''

"\nk : le nombre d'itération de notre modèle \nn : le nombre d'itération avant de diminuer epsilon\n"

In [ ]:
Q_MC = np.zeros((len(S),9))
pi_MC = Pi.copy()
pi_MC , Q_MC = Monte_Carlo(pi_MC,Q_MC,k = 15000,n=15000)

In [ ]:
Q_Sarsa = np.zeros((len(S),9))
pi_Sarsa = Pi.copy()

In [ ]:
pi_Sarsa , Q_Sarsa = Sarsa(pi_Sarsa,Q_Sarsa,k = 15000, n = 10000)

In [ ]:
Q_Sarsa_2 = np.zeros((len(S),9))
pi_Sarsa_2 = Pi.copy()
pi_Sarsa_2 , Q_Sarsa_2 = Sarsa(pi_Sarsa_2,Q_Sarsa_2,k = 10000, n = 10000)

In [ ]:
pi_MC[0]

array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
       0.11111111, 0.11111111, 0.11111111, 0.11111111])

## Match contre l'IA

In [ ]:
def match(pi):
  E = [0,0,0,
        0,0,0,
        0,0,0]
  print_etat(E)
  while(not(etat_terminal(E))):
    a = predict(E,pi)
    E = etat_action(E,a,1)
    print_etat(E)
    if etat_final(E):
      print("le robot gagne")
      return 0
    if etat_terminal(E):
      print("égalité")
      return 0
    action = input("votre coup : ")
    E = etat_action(E,int(action),-1)
    print_etat(E)

In [ ]:
match(pi_MC)


0 0 0            0 1 2
0 0 0            3 4 5
0 0 0            6 7 8
--------

0 0 1            0 1 2
0 0 0            3 4 5
0 0 0            6 7 8
--------
votre coup : 4

0 0 1            0 1 2
0 -1 0            3 4 5
0 0 0            6 7 8
--------

0 1 1            0 1 2
0 -1 0            3 4 5
0 0 0            6 7 8
--------
votre coup : 8

0 1 1            0 1 2
0 -1 0            3 4 5
0 0 -1            6 7 8
--------

0 1 1            0 1 2
0 -1 0            3 4 5
1 0 -1            6 7 8
--------
votre coup : 0

-1 1 1            0 1 2
0 -1 0            3 4 5
1 0 -1            6 7 8
--------
